In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate, FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
import os
    

chat = ChatOpenAI(
  temperature = 0.1,
  streaming = True,
  callbacks=[StreamingStdOutCallbackHandler()],
  openai_api_key=os.getenv("OPEN_API_KEY")
)

In [12]:

examples = [
  {"movie": "Aliens",
   "answer": """
   Here is what I know
   Director: Ridley Scott
   Actors: Sigourney Weaver, Tom Skerritt,John Hurt
   Budget: 10 million ($)
   Box Office: 104 million ($)
   Genre: Action
   Sysnopsis: Aliens are attacking the Earth.
   """
  },
  {"movie": "Saving Private Ryan",
   "answer": """
   I know this:
   Director: Steven Spielberg
   Actors: Tom Hanks, Tom Sizemore, Edward Burns
   Budget: 70 million ($)
   Box Office: 482 million ($)
   Genre: Action
   Sysnopsis: Saving Private Ryan is a movie about World War II.
   """
  },
  {"movie": "The Matrix",
   "answer": """
   Information about The Matrix:
   Director: The Wachowskis
   Actors: Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss
   Budget: 63 million ($)
   Box Office: 461 million ($)
   Genre: Action
   Sysnopsis: The Matrix is a movie about a computer simulation.
   """
  },
]

In [13]:

example_prompt = PromptTemplate.from_template(
    "Human: What do you know about {movie}? \n AI: {answer}"
)

example_prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
  suffix="Human: What do you know about {movie}?",
  input_variables=["movie"],
)

chain = example_prompt | chat

print(chain.invoke({"movie": "Parasite"}))

AI: 
   Here is what I know about Parasite:
   Director: Bong Joon-ho
   Actors: Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong
   Budget: 11 million ($)
   Box Office: 266 million ($)
   Genre: Thriller
   Sysnopsis: Parasite is a South Korean film about a poor family who infiltrates a wealthy household.content='AI: \n   Here is what I know about Parasite:\n   Director: Bong Joon-ho\n   Actors: Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong\n   Budget: 11 million ($)\n   Box Office: 266 million ($)\n   Genre: Thriller\n   Sysnopsis: Parasite is a South Korean film about a poor family who infiltrates a wealthy household.'


In [14]:

example_prompt = ChatPromptTemplate.from_messages(
  [
    ("human", "What do you know about {movie}?"),
    ("ai", "{answer}"),
  ]
)

chat_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a movie expert. You have to give me the information about the movie."),
    chat_prompt,
    ("human", "What do you know about {movie}?"),
  ]
)

chain = final_prompt | chat

print(chain.invoke({"movie": "Parasite"}))

Information about Parasite:
   Director: Bong Joon-ho
   Actors: Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong
   Budget: 11 million ($)
   Box Office: 266 million ($)
   Genre: Thriller/Drama
   Synopsis: Parasite is a South Korean film that follows the story of a poor family who schemes to work for a wealthy family by posing as unrelated, highly qualified individuals. The film explores themes of class struggle and social inequality. It won numerous awards, including the Academy Award for Best Picture in 2020.content='Information about Parasite:\n   Director: Bong Joon-ho\n   Actors: Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong\n   Budget: 11 million ($)\n   Box Office: 266 million ($)\n   Genre: Thriller/Drama\n   Synopsis: Parasite is a South Korean film that follows the story of a poor family who schemes to work for a wealthy family by posing as unrelated, highly qualified individuals. The film explores themes of class struggle and social inequality. It won numerous awards, including the 